# 1. Preparation for Gridded ICESat-2 ATL08 

Jian Li, PhD  
jian.li@nasa.gov

 - Aggregate ICESat-2 ATL08 20m point observations into CMIP6 bioclimatic grids at ~1km resoltuion

### Compiling observations of current forest canopy height from spaceborne lidar
We compiled spaceborne lidar observations of current forest canopy height from ICESat-2 Land - Vegetation Along-Track Products (ATL08 v5) 20 m segment data. These data provide a broad sample of vegetation heights useful for sampling the broad domain of the boreal forest 3,4. These observations were gathered from granules acquired seasonally during June-September in 2018-2022 for North America (45 N - 75 N, 50 W - 160 W). We used ATL08 data quality flags and values to limit the set of ATL08 observations to those most suited for analysis of boreal vegetation. As such, we filtered these observations to include only those associated with:
1. strong beams, 
2. snow-free land surface, 
3. cloud-free conditions,
4. night and low light (solar elevation angle < 5 degrees), 
5. a valid 98th percentile height range  ≤ 30 meters & below ATL08 land-cover-specific height thresholds (Table S1), 
6. a height difference from reference elevation < 25 meters, 
7. a total vertical geolocation error due to ranging and local slope < 2.5 meters

Within the training and testing domain we created a ‘canopy height flag’ to identify and separate assumed erroneous reference ATL08 observations of canopy height. Using this flag, we set reference ATL08 observations to 0 m. This step was helpful in preparing a training dataset that included areas currently featuring no vegetation height, while also mitigating the bias in canopy heights present in filtered ATL08 data in the broad area of bare ground and tundra at the coldest northern edge of our study domain by reducing the overestimation of current and future vegetation structure in currently non-forested extents. To mitigate spurious observations at the high end, we also applied a height cap, by assigning a height of 30 m to those observations greater than this value. This yielded 19,875,592 point-based ICESat-2 ATL08 observations of canopy height across the full training and testing domain. These filtered and flagged reference canopy height observations were then gridded to 1 km and aggregated by the maximum observation, generating a set of 1,082,746 ICESat-2 ATL08 gridded canopy height observations that approximates the 98th percentile of height for the tallest forests in each grid cell for circa 2020. A portion (80%) of these observations were used to train a model for predicting canopy height based on a set of predictor variables described below.

In [1]:
import geopandas as gpd
import rioxarray as rxr
import numpy as np
import xarray as xr
import sys
import os
import glob
from shapely.geometry import box

In [3]:
def create_cells(xarr, yarr, cell_sz):
    grid_cells = []
    for x0 in xarr:
        for y0 in yarr:
            x1 = x0+cell_sz
            y1 = y0+cell_sz
            grid_cells.append(box(x0, y0, x1, y1))
    return gpd.GeoDataFrame(grid_cells, columns=['geometry'], crs='EPSG:4326')

In [4]:
# ATL08 bounding box corner coords
tl = (-169, 75)
lr = (-50, 45)

In [5]:
# read any bioclim & extract spatial info
biofn = "/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim/wc2.0_30s_bio/wc2.0_bio_30s_02.tif"
ds = rxr.open_rasterio(biofn)
csize = ds.rio.resolution()[0] # cell size of raster
lons = ds.x.values
lats = ds.y.values

In [6]:
# subset to ATL08 bounding box
clip = ds.rio.clip_box(tl[0], lr[1], lr[0], tl[1])
raster = xr.zeros_like(clip)+np.nan
raster_dims = raster.shape
raster_dims

(1, 3600, 14280)

In [7]:
# co-locate ATL08 corner to bioclim lat-lon coords
xid_min = list(lons).index(min(list(lons), key=lambda x : abs(x-tl[0])))
xid_max = xid_min+len(raster.x)-1
list(lons)[xid_min], list(lons)[xid_max]

(-169.00416666666666, -50.01250000000002)

In [8]:
yid_min = list(lats).index(min(list(lats), key=lambda x : abs(x-tl[1])))
yid_max = yid_min+len(raster.y)-1
list(lats)[yid_min], list(lats)[yid_max]

(75.00416666666666, 45.0125)

In [9]:
# break into pataches
xbin_size = 320
ybin_size = 85

In [10]:
xid_s = list(range(xid_min, xid_max, xbin_size))
yid_s = list(range(yid_min, yid_max, ybin_size))
print(len(xid_s),len(yid_s))

45 43


In [24]:
# start mapping ATL08 to bioclim grids
#atl08_path = "/explore/nobackup/people/pmontesa/userfs02/projects/ilab/boreal_height_cmip/atl08_boreal_na_20m_intersect_filt_2018_2019.parquet"
#atl08_path = "/explore/nobackup/people/pmontesa/userfs02/projects/ilab/boreal_height_cmip/atl08_boreal_na_20m_intersect_filt_2020_2020.parquet"
#atl08_path = "/explore/nobackup/people/pmontesa/userfs02/projects/ilab/boreal_height_cmip/atl08_boreal_na_20m_intersect_filt_2021_2021.parquet"
atl08_path = "/explore/nobackup/people/pmontesa/userfs02/projects/ilab/boreal_height_cmip/atl08_boreal_na_20m_intersect_filt_2022_2022.parquet"
c_gdf = gpd.read_parquet(atl08_path)

In [25]:
# sample ATL08 data
c_gdf.head()

,id_unique,lon,lat,h_can_20m,lon_100m,lat_100m,h_can,h_max_can,h_te_best,rh25,...,seg_landcov,seg_cover,sol_el,dt,granule_name,y,m,doy,height_class,geometry
0,70W2593000-49N7455550-2022619-1,-70.259247,49.745197,12.496307,-70.2593,49.745556,12.496307,12.762238,399.208191,2.536591,...,111,100,-16.095457,b'2022-06-19T03:58:39.000000Z',ATL08_20220619035009_13451502_005_02.h5,2022,6,170,h_can,POINT (-70.25925 49.74520)
1,70W2593000-49N7455550-2022619-2,-70.259277,49.745377,10.244843,-70.2593,49.745556,12.496307,12.762238,399.208191,2.536591,...,111,100,-16.095457,b'2022-06-19T03:58:39.000000Z',ATL08_20220619035009_13451502_005_02.h5,2022,6,170,h_can,POINT (-70.25928 49.74538)
2,70W2593000-49N7455550-2022619-3,-70.259300,49.745556,12.762238,-70.2593,49.745556,12.496307,12.762238,399.208191,2.536591,...,111,100,-16.095457,b'2022-06-19T03:58:39.000000Z',ATL08_20220619035009_13451502_005_02.h5,2022,6,170,h_can,POINT (-70.25930 49.74556)
3,70W2593000-49N7455550-2022619-4,-70.259331,49.745735,11.105286,-70.2593,49.745556,12.496307,12.762238,399.208191,2.536591,...,111,100,-16.095457,b'2022-06-19T03:58:39.000000Z',ATL08_20220619035009_13451502_005_02.h5,2022,6,170,h_can,POINT (-70.25933 49.74574)
4,70W2593000-49N7455550-2022619-5,-70.259361,49.745914,12.199280,-70.2593,49.745556,12.496307,12.762238,399.208191,2.536591,...,111,100,-16.095457,b'2022-06-19T03:58:39.000000Z',ATL08_20220619035009_13451502_005_02.h5,2022,6,170,h_can,POINT (-70.25936 49.74591)


In [26]:
# extract ATL08 points geometry information for later usage
c_points = c_gdf['geometry']
len(c_points)

4248609

In [27]:
%%time

for x1 in xid_s:
    for y1 in yid_s:
        x2 = x1+xbin_size
        if x2 > xid_max : x2 = xid_max
        y2 = y1+ybin_size
        if y2 > yid_max : y2 = yid_max

        #check if there is intersection with atl08
        n = len(c_points.cx[lons[x1]:lons[x2], lats[y1]:lats[y2]])
        if n == 0:
            pass
        else:
            cell = create_cells(lons[x1:x2], lats[y1:y2], csize)
            gdf_sub = c_gdf[['h_can_20m', 'geometry']].cx[lons[x1]:lons[x2], lats[y1]:lats[y2]]
            merged = gpd.sjoin(gdf_sub, cell, how='left', op='within')
            dissolve_med = merged.dissolve(by='index_right', aggfunc='mean')
            cell.loc[dissolve_med.index, 'h_can_20m'] = dissolve_med['h_can_20m'].values
            h_arr= cell['h_can_20m'].values.reshape((y2-y1), (x2-x1), order='F')
            raster.data[0, (y1-yid_min):(y2-yid_min), (x1-xid_min):(x2-xid_min)] = h_arr
    
fout = os.path.basename(atl08_path).split('.')[0]
fout = fout+"_mean_raster.tif"
raster.rio.to_raster(fout)      

490 1319 1639 2819 2904


/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = fn(*args, **kwargs)


2174 1319 1639 2904 2989


/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = fn(*args, **kwargs)


273 1319 1639 2989 3074


...

8589 14759 15079 4604 4689


/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = fn(*args, **kwargs)


4688 14759 15079 4689 4774


/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2417: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  result = fn(*args, **kwargs)


CPU times: user 11min 5s, sys: 1min 34s, total: 12min 40s
Wall time: 11min 53s


## Aggregate ATL08 rasters along time
Using two agg func: nanmean & max

In [16]:
fs = sorted(glob.glob("atl08_boreal*"))
fs

['atl08_boreal_na_20m_intersect_filt_2018_2019_mean_raster.tif',
 'atl08_boreal_na_20m_intersect_filt_2020_2020_mean_raster.tif',
 'atl08_boreal_na_20m_intersect_filt_2021_2021_mean_raster.tif',
 'atl08_boreal_na_20m_intersect_filt_2022_2022_mean_raster.tif']

In [18]:
ds_list = [rxr.open_rasterio(f) for f in fs]
ds_combine = xr.concat(ds_list, dim="band")
ds_combine

<xarray.DataArray (band: 4, y: 3600, x: 14280)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * band         (band) int64 1 1 1 1
  * x            (x) float64 -169.0 -169.0 -169.0 -169.0 ... -50.02 -50.01 -50.0
  * y            (y) float64 75.0 74.99 74.98 74.97 ... 45.03 45.02 45.01 45.0
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [19]:
# reduce with max
ds_max = ds_combine.max(dim="band")
fo = "atl08_boreal_na_20m_intersect_filt_2018_2022_max_raster.tif"
ds_max.rio.to_raster(fo)